In [1]:
# DEPENDENCIES
import pandas as pd

# READ SCHOOL AND STUDENT DATA INTO PANDAS DATAFRAMES
school_data_df = pd.read_csv("Resources/schools_complete.csv")
student_data_df = pd.read_csv("Resources/students_complete.csv")

# CLEAN STUDENTS NAMES
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
for word in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(word,"")

student_data_df.head(3)

/Users/jaz/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60


In [2]:
# 1-4. REPLACE THOMAS HS 9TH GRADERS' READING AND MATH SCORES WITH NaN
import numpy as np

In [3]:
ths_idx = student_data_df["school_name"] == "Thomas High School"
nineth_idx = student_data_df["grade"] == "9th"
ths_nineth_idx = ths_idx & nineth_idx
student_data_df.loc[ths_nineth_idx, "reading_score"] = np.nan
student_data_df.loc[ths_nineth_idx, "math_score"] = np.nan

student_data_df.tail(10)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39160,39160,Katie Weaver,F,11th,Thomas High School,89.0,86.0
39161,39161,April Reyes,F,10th,Thomas High School,70.0,84.0
39162,39162,Derek Weeks,M,12th,Thomas High School,94.0,77.0
39163,39163,John Reese,M,11th,Thomas High School,90.0,75.0
39164,39164,Joseph Anthony,M,9th,Thomas High School,NaN,NaN
39165,39165,Donna Howard,F,12th,Thomas High School,99.0,90.0
39166,39166,Dawn Bell,F,10th,Thomas High School,95.0,70.0
39167,39167,Rebecca Tanner,F,9th,Thomas High School,NaN,NaN
39168,39168,Desiree Kidd,F,10th,Thomas High School,99.0,90.0
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95.0,75.0


In [4]:
# CHECK STUDENT DATA FOR NaNs
student_data_df.isnull().sum()

Student ID         0
student_name       0
gender             0
grade              0
school_name        0
reading_score    461
math_score       461
dtype: int64

In [5]:
# MERGE SCHOOL AND STUDENT DATA INTO ONE CLEAN DATAFRAME
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66.0,79.0,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94.0,61.0,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90.0,60.0,0,District,2917,1910635
3,3,Richard Scott,M,12th,Huang High School,67.0,58.0,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97.0,84.0,0,District,2917,1910635


In [6]:
# COMPLETE THE SCHOOL DISTRICT ANALYSIS
# CALCULATE STUDENT AND SCHOOL TOTALS
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()

# CALCULATE THE TOTAL BUDGET
total_budget = school_data_df["budget"].sum()

In [7]:
# CALCULATE AVERAGE SCORES
average_reading_score = school_data_complete_df["reading_score"].mean()
average_math_score = school_data_complete_df["math_score"].mean()

In [8]:
# CALCULATE TOTAL STUDENT COUNT (EXCLUDING THS 9TH GRADERS)
ths_nineth_graders = student_data_df["reading_score"].isnull().sum()
student_count = student_count - ths_nineth_graders

In [9]:
# CALCULATE PASSING RATES
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]

In [10]:
# CALCULATE PASSING PERCENTAGES
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [11]:
# CALCULATE STUDENTS PASSING READING AND MATH
passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)
                                               & (school_data_complete_df["reading_score"] >= 70)]
overall_passing_math_reading_count = passing_math_reading["student_name"].count()
# CALCULATE OVERALL PASSING PERCENTAGE
overall_passing_percentage = overall_passing_math_reading_count / float(student_count) * 100

In [12]:
# CREATE A DISTRICT SUMMARY DATAFRAME
district_summary_df = pd.DataFrame(
          [{"Total Schools": school_count, 
          "Total Students": student_count, 
          "Total Budget": total_budget,
          "Average Math Score": average_math_score, 
          "Average Reading Score": average_reading_score,
          "% Passing Math": passing_math_percentage,
         "% Passing Reading": passing_reading_percentage,
        "% Overall Passing": overall_passing_percentage}])

# FORMAT THE "TOTAL STUDENTS" TO HAVE THE COMMA FOR A THOUSANDS SEPARATOR
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
# FORMAT THE "TOTAL BUDGET" TO HAVE THE COMMA FOR A THOUSANDS SEPARATOR, A DECIMAL SEPARATOR AND A "$"
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
# FORMAT THE COLUMNS
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)
district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.1f}".format)
district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.1f}".format)
district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.1f}".format)
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"38,709","$24,649,428.00",78.9,81.9,74.8,85.7,64.9


In [13]:
# SCHOOL SUMMARY
# DETERMINE SCHOOL TYPES
per_school_types = school_data_df.set_index(["school_name"])["type"]

# CALCULATE TOTAL STUDENT COUNT
per_school_counts = school_data_complete_df["school_name"].value_counts()

# CALCULATE TOTAL SCHOOL BUDGET AND PER CAPITA SPENDING
per_school_budget = school_data_complete_df.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

# CALCULATE AVERAGE SCORES
per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

# CALCULATE PASSING SCORES WITH A FILTERED DATAFRAME
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]

# CALCULATE STUDENTS PASSING MATH AND READING BY SCHOOL
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]

# CALCULATE PERCENTAGE OF PASSING MATH AND READING SCORES BY SCHOOL
per_school_passing_math = per_school_passing_math / per_school_counts * 100
per_school_passing_reading = per_school_passing_reading / per_school_counts * 100

# CALCULATE OVERALL PASSING PERCENTAGE
per_passing_math_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)
                                               & (school_data_complete_df["math_score"] >= 70)]
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]
per_overall_passing_percentage = per_passing_math_reading / per_school_counts * 100

In [14]:
# CREATE THE SCHOOL SUMMARY DATAFRAME
per_school_summary_df = pd.DataFrame({
    "School Type": per_school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": per_school_passing_math,
    "% Passing Reading": per_school_passing_reading,
    "% Overall Passing": per_overall_passing_percentage})

# FORMAT THE TOTAL SCHOOL BUDGET AND THE PER STUDENT BUDGET
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)
per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [17]:
# 5. GET THE NUMBER OF 10TH-12TH GRADERS FROM THOMAS HS
ths_student_count = school_data_complete_df["school_name"] == "Thomas High School"
ths_clean_student_total = ths_student_count.sum() - ths_nineth_graders
ths_clean_student_total

1174

In [18]:
# 6. GET ALL THE STUDENTS PASSING MATH FROM THS USING LOC METHOD AND A NEW DATAFRAME
ths_passing_math = school_data_complete_df.loc[(student_data_df["school_name"]=="Thomas High School") 
                    & (school_data_complete_df["grade"]!="9th") 
                    & (school_data_complete_df["math_score"]>=70)]
ths_passing_math.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
37535,37535,Norma Mata,F,10th,Thomas High School,76.0,76.0,14,Charter,1635,1043130
37536,37536,Cody Miller,M,11th,Thomas High School,84.0,82.0,14,Charter,1635,1043130
37541,37541,Eric Stevens,M,10th,Thomas High School,80.0,76.0,14,Charter,1635,1043130
37542,37542,Elizabeth Bennett,F,11th,Thomas High School,91.0,94.0,14,Charter,1635,1043130
37544,37544,Jacqueline Harris,F,10th,Thomas High School,71.0,92.0,14,Charter,1635,1043130


In [19]:
# 7. GET ALL THE STUDENTS PASSING READING FROM THS USING LOC METHOD AND A NEW DATAFRAME
ths_passing_reading = school_data_complete_df.loc[(student_data_df["school_name"]=="Thomas High School") 
                    & (school_data_complete_df["grade"]!="9th") 
                    & (school_data_complete_df["reading_score"]>=70)]

In [20]:
# 8. GET ALL THE STUDENTS PASSING MATH AND READING FROM THS USING LOC METHOD AND A NEW DATAFRAME
ths_passing = school_data_complete_df.loc[(student_data_df["school_name"]=="Thomas High School") 
                    & (school_data_complete_df["grade"]!="9th") 
                    & (school_data_complete_df["reading_score"]>=70) 
                    & (school_data_complete_df["math_score"]>=70)]

In [21]:
# 9. CALCULATE PERCENTAGE OF 10TH-12TH GRADERS PASSING MATH FROM THS
ths_passing_math_count = ths_passing_math["Student ID"].count()
ths_passing_math = ths_passing_math_count / float(ths_clean_student_total) * 100
ths_passing_math

93.18568994889267

In [22]:
# 10. CALCULATE PERCENTAGE OF 10TH-12TH GRADERS PASSING READING FROM THS
ths_passing_reading_count = ths_passing_reading["Student ID"].count()
ths_passing_reading = ths_passing_reading_count / float(ths_clean_student_total) * 100
ths_passing_reading

97.01873935264055

In [23]:
# 11. CALCULATE THE OVERALL PASSING PERCENTAGE OF 10TH-12TH GRADERS FROM THS
ths_overall_passing_count = ths_passing["Student ID"].count()
ths_overall_passing = ths_overall_passing_count / float(ths_clean_student_total) * 100
ths_overall_passing

90.63032367972743

In [24]:
# 12. REPLACE THE PASSING MATH PERCENTAGE FOR THS IN THE per_school_summary_df
per_school_summary_df.loc[["Thomas High School"], ["% Passing Math"]] = ths_passing_math

In [25]:
# 13. REPLACE THE PASSING READING PERCENTAGE FOR THS IN THE per_school_summary_df
per_school_summary_df.loc[["Thomas High School"], ["% Passing Reading"]] = ths_passing_reading

In [26]:
# 14. REPLACE THE OVERALL PASSING PERCENTAGE FOR THS IN THE per_school_summary_df
per_school_summary_df.loc[["Thomas High School"], ["% Overall Passing"]] = ths_overall_passing

In [30]:
# SHOW UPDATED SCHOOL SUMMARY DF
per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [28]:
# HIGH AND LOW PERFORMING SCHOOLS
high_perf_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)
high_perf_schools.head()

low_perf_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=True)
low_perf_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [31]:
# MATH AND READING SCORES BY GRADE
# SEPARATE STUDENTS BY GRADE LEVEL
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

In [32]:
# GROUP BY SCHOOL NAME AND GET AVERAGE MATH AND READING SCORES
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [33]:
# COMBINE EACH SERIES FOR AVERAGE MATH SCORES BY SCHOOL INTO SINGLE DATA FRAME
math_scores_by_grade = pd.DataFrame({
               "9th": ninth_grade_math_scores,
               "10th": tenth_grade_math_scores,
               "11th": eleventh_grade_math_scores,
               "12th": twelfth_grade_math_scores})
math_scores_by_grade.head()

# COMBINE EACH SERIES FOR AVERAGE READING SCORES BY SCHOOL INTO SINGLE DATA FRAME
reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_scores,
              "10th": tenth_grade_reading_scores,
              "11th": eleventh_grade_reading_scores,
              "12th": twelfth_grade_reading_scores})
reading_scores_by_grade.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


In [34]:
# FORMAT EACH GRADE COLUMN FOR MATH SCORES
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)
# MAKE SURE COLUMNS ARE IN CORRECT ORDER
math_scores_by_grade = math_scores_by_grade[["9th", "10th", "11th", "12th"]]
# REMOVE INDEX NAME
math_scores_by_grade.index.name = None
math_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4


In [35]:
 # FORMAT EACH GRADE COLUMN FOR READING SCORES
reading_scores_by_grade["9th"] = reading_scores_by_grade["9th"].map("{:,.1f}".format)
reading_scores_by_grade["10th"] = reading_scores_by_grade["10th"].map("{:,.1f}".format)
reading_scores_by_grade["11th"] = reading_scores_by_grade["11th"].map("{:,.1f}".format)
reading_scores_by_grade["12th"] = reading_scores_by_grade["12th"].map("{:,.1f}".format)
# MAKE SURE COLUMNS ARE IN CORRECT ORDER
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
# REMOVE INDEX NAME
reading_scores_by_grade.index.name = None
reading_scores_by_grade.head()

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0


In [36]:
# SCORES BY SCHOOL SPENDING
per_school_capita.describe()
# ESTABLISH SPENDING BINS AND GROUP NAMES
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$584", "$585-629", "$630-644", "$645-675"]
# CATEGORIZE SPENDING BASED ON THE BINS
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)
per_school_summary_df.head(3)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$630-644
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$584
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644


In [37]:
# CALCULATE AVERAGES FOR THE COLUMNS
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]
overall_passing_spending

Spending Ranges (Per Student)
<$584       90.369459
$585-629    90.216324
$630-644    66.059112
$645-675    53.526855
Name: % Overall Passing, dtype: float64

In [38]:
# CREATE THE SCHOOL SPENDING DATAFRAME
spending_summary_df = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.455399,83.933814,93.460096,96.610877,90.369459
$585-629,83.599686,83.885211,94.230858,95.900287,90.216324
$630-644,79.067990,81.899294,75.653798,86.058214,66.059112
$645-675,76.997210,81.027843,66.164813,81.133951,53.526855


In [39]:
# Format the DataFrame
spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)
spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,93,97,90
$585-629,83.6,83.9,94,96,90
$630-644,79.1,81.9,76,86,66
$645-675,77.0,81.0,66,81,54


In [40]:
# SCORES BY SCHOOL SIZE
# ESTABLISH THE BINS AND GROUP NAMES
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
# CATEGORIZE SPENDING BASED ON THE BINS
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)
per_school_summary_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$630-644,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$584,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-644,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-644,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$630-644,Medium (1000-2000)


In [41]:
# CALCULATE AVERAGES FOR THE COLUMNS
size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

In [42]:
# CREATE SIZE SUMMARY DATAFRAME 
size_summary_df = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.361201,83.873869,93.582398,96.732654,90.557997
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [43]:
# FORMAT THE SIZE SUMMARY DATAFRAME
size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)
size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.8,83.9,94,96,90
Medium (1000-2000),83.4,83.9,94,97,91
Large (2000-5000),77.7,81.3,70,83,58


In [44]:
# SCORES BY SCHOOL TYPE
# CALCULATE AVERAGES FOR THE COLUMNS
type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

In [45]:
# CREATE SCHOOL TYPE DATAFRAME
type_summary_df = pd.DataFrame({
          "Average Math Score" : type_math_scores,
          "Average Reading Score": type_reading_scores,
          "% Passing Math": type_passing_math,
          "% Passing Reading": type_passing_reading,
          "% Overall Passing": type_overall_passing})
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.465425,83.902315,93.610020,96.550223,90.392533
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [46]:
# FORMAT THE SCHOOL TYPE DATAFRAME
type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)
type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
